In [1]:

#!python3 -m pip install pytorch_lightning
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model import LSTM_Model
from pytorch_lightning import seed_everything, LightningModule, Trainer
from torch import save
from pytorch_lightning.callbacks import EarlyStopping
from torch.utils.data import DataLoader
import neptune.new as neptune
from pytorch_lightning.loggers import NeptuneLogger
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint
from BCM_dataset_v2 import bcmDataset, concat_train_test_datasets
import glob
import os
import sys
from collect_dataset import*

Dataset version: 5


In [2]:

ft_datasets = create_ft_datasets_da()
                                                   

#loso_datasets = create_loso_datasets()


Fint-tuning datasets


train

0
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-34-16
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-52-7
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-15-6
bcm_behaviour_data_multi_subject/subject2/2022-11-09_13-56-0
bcm_behaviour_data_multi_subject/subject3/2022-11-09_15-47-1
bcm_behaviour_data_multi_subject/subject3/2022-11-09_16-26-13
bcm_behaviour_data_multi_subject/subject3/2022-11-09_15-27-14
bcm_behaviour_data_multi_subject/subject3/2022-11-09_16-7-9
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-15-6speed
bcm_behaviour_data_multi_subject/subject2/2022-11-09_13-56-0speed
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-34-16speed
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-52-7speed
bcm_behaviour_data_multi_subject/subject3/2022-11-09_15-47-1speed
bcm_behaviour_data_multi_subject/subject3/2022-11-09_16-26-13speed
bcm_behaviour_data_multi_subject/subject3/2022-11-09_15-27-14speed
bcm_behaviou

In [3]:
seed = 555
seed_everything(seed)
    
neptune_logger = NeptuneLogger(
    project="NTLAB/BCM-activity-classification",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYTA4NzcxMy1lYmQ2LTQ3NTctYjRhNC02Mzk1NjdjMWM0NmYifQ==",
    source_files=["train_model.ipynb", "model.py", "BCM_dataset_v2.py"]
)

checkpoint_callback = ModelCheckpoint(dirpath = 'checkpoints/', filename = 'latest_checkpoint')

model = LSTM_Model(ft_datasets[1], window_size = 3, stride = 1 ,lstm_hidden_size = 128, seed = seed, temporal_cutout = False)

trainer = Trainer(max_epochs=40, 
                min_epochs=1, 
                auto_lr_find=False, 
                auto_scale_batch_size=False,
                enable_checkpointing=True, 
                accelerator="gpu", 
                devices = 1, 
                logger=neptune_logger,
                callbacks = [checkpoint_callback])
trainer.tune(model)
trainer.fit(model)
        
neptune_logger.experiment["metadata/augmentation"].log("best")

# Find the latest checkpoint
list_of_checkpoints = glob.glob('checkpoints/*') 
latest_checkpoint = max(list_of_checkpoints, key = os.path.getctime) 
print(latest_checkpoint) # Sanity check
    
    
    
trainer_finetune = Trainer(max_epochs=80, 
                    min_epochs=100, 
                    auto_lr_find=False, 
                    auto_scale_batch_size=False,
                    enable_checkpointing=True, 
                    accelerator="gpu", 
                    devices = 1, 
                    logger=neptune_logger,
                    callbacks = [checkpoint_callback])
trainer_finetune.tune(model)
    
# Create a dataloader from the new subject specific data

dataset = ft_datasets[1]

dataset_finetune, _ = concat_train_test_datasets([dataset[2],dataset[2]], window_size = 3, stride = 1)
    
neptune_logger.experiment["metadata/train_set_length"].log(len(dataset_finetune))
    
finetuning_dataloader = DataLoader(dataset_finetune, batch_size=32, shuffle=True, num_workers = 8)
    
trainer_finetune.fit(model, finetuning_dataloader, ckpt_path=latest_checkpoint)
        
        
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(f"trained_models/ft_final_ft_file_2022-11-09_14-34-16") # Save
        
        
# Stop logging
neptune_logger.experiment.stop()
        

Global seed set to 555


_____________________
<class 'list'>
_____________________
Validation set
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-52-7/0.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-52-7/1.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-52-7/2.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-52-7/3.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-52-7/4.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-15-6/0.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-15-6/1.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-15-6/2.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-15-6/3.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_14-15-6/4.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_13-56-0/0.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_13-56-0/1.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_13-56-0/2.npy
bcm_behaviour_data_multi_subject/subject2/2022-11-09_13

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/aron/.local/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/aron/BCM_activity_classification/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


https://app.neptune.ai/NTLAB/BCM-activity-classification/e/BCMAC-1367
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.



  | Name         | Type             | Params
--------------------------------------------------
0 | lstm         | LSTM             | 149 K 
1 | flatten      | Flatten          | 0     
2 | fc           | Linear           | 1.3 K 
3 | output       | Sigmoid          | 0     
4 | sm           | Softmax          | 0     
5 | loss         | CrossEntropyLoss | 0     
6 | accuracy     | Accuracy         | 0     
7 | val_accuracy | Accuracy         | 0     
--------------------------------------------------
150 K     Trainable params
0         Non-trainable params
150 K     Total params
0.603     Total estimated model params size (MB)


...

{'progress_bar': {'val_loss': 1.4401581287384033}, 'log': {'val_loss': 1.4401581287384033}, 'val_loss': 1.4401581287384033}
Accuracy: 0.8872794508934021
Accuracy: 0.9660167098045349


/home/aron/.local/lib/python3.9/site-packages/torch/jit/_recursive.py:621: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(mod, name, None)
/home/aron/.local/lib/python3.9/site-packages/torch/jit/_recursive.py:708: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(nn_module, name, None)


Shutting down background jobs, please wait a moment...
Done!


/home/aron/.local/lib/python3.9/site-packages/torch/jit/_recursive.py:508: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(nn_module, name, None)
/home/aron/.local/lib/python3.9/site-packages/torch/jit/_recursive.py:591: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(nn_module, name, None)
Waiting for the remaining 64 operations to synchronize with Neptune. Do not kill this process.
Error occurred during asynchronous operation processing: Cannot delete training/model/checkpoints/latest_checkpoint: Attribute does not exist


All 64 operations synced, thanks for waiting!


In [4]:


#model_scripted = torch.jit.script(model) # Export to TorchScript
#model_scripted.save("trained_models/oct_31") # Save